In [1]:
import pandas as pd
from tqdm import tqdm
from experiments import Experiment

In [2]:
df = pd.read_csv('../data/german_credit.csv')

In [3]:
features = [
'status_checking_account',
'duration_in_month',
'credit_history',
'purpose',
'savings',
'employement_since',
'installment_rate',
'debters',
'resident_since',
'property',
'age',
'other_installments',
'housing',
'num_credits',
'job',
'num_liable',
'telephone',
'foreign_worker'
]
protected_attribute = "gender"
random_thresholds = [0, 0.1, 0.2, 0.3, 0.4, 0.5]
task_types =  ["is_good_loan", "is_high_credit"]
model_types = ["logistic", "gbm", "nn", "svm", "tree"]
n_train = 800
n_test = 200

In [5]:
data_scales = [10, 9, 8, 7, 6, 5, 4, 3, 2]
random_seeds = list(range(5))

results = []

for data_scale in tqdm(data_scales):
    for random_seed in tqdm(random_seeds):
        exp = Experiment(df, features, protected_attribute, task_types,
                             model_types, random_thresholds, n_train, n_test, random_seed, data_scale)

        exp.pretrain_models()
        results.append(exp.experiment_baseline())
        results.append(exp.experiment_tasks())
        results.append(exp.experiment_models())
        
        results.append(exp.experiment_partitions(min(5, data_scale)))
results = pd.concat(results)
results.to_csv("german_credit_results.csv", index=False)

  0%|                                                     | 0/5 [00:00<?, ?it/s]

is_good_loan logistic
is_good_loan gbm
is_good_loan nn
is_good_loan svm
is_good_loan tree
is_high_credit logistic
is_high_credit gbm
is_high_credit nn
is_high_credit svm
is_high_credit tree
Running Baseline Experiment
Running Tasks Experiment
Running Models Experiment
Running Data Partitions Experiment
is_good_loan logistic
is_good_loan gbm
is_good_loan nn
is_good_loan svm
is_good_loan tree
is_high_credit logistic
is_high_credit gbm
is_high_credit nn
is_high_credit svm



 20%|█████████                                    | 1/5 [00:03<00:15,  3.91s/it]

is_high_credit tree
is_good_loan logistic
is_good_loan gbm
is_good_loan nn
is_good_loan svm
is_good_loan tree
is_high_credit logistic
is_high_credit gbm
is_high_credit nn
is_high_credit svm
is_high_credit tree
Running Baseline Experiment
Running Tasks Experiment
Running Models Experiment
Running Data Partitions Experiment
is_good_loan logistic
is_good_loan gbm
is_good_loan nn
is_good_loan svm
is_good_loan tree
is_high_credit logistic
is_high_credit gbm
is_high_credit nn
is_high_credit svm



 40%|██████████████████                           | 2/5 [00:07<00:11,  3.91s/it]

...

is_high_credit tree
is_good_loan logistic
is_good_loan gbm
is_good_loan nn
is_good_loan svm
is_good_loan tree
is_high_credit logistic
is_high_credit gbm
is_high_credit nn
is_high_credit svm
is_high_credit tree
Running Baseline Experiment
Running Tasks Experiment
Running Models Experiment
Running Data Partitions Experiment
is_good_loan logistic
is_good_loan gbm
is_good_loan nn
is_good_loan svm
is_good_loan tree
is_high_credit logistic
is_high_credit gbm
is_high_credit nn
is_high_credit svm



 80%|████████████████████████████████████         | 4/5 [00:15<00:03,  3.84s/it]

is_high_credit tree
is_good_loan logistic
is_good_loan gbm
is_good_loan nn
is_good_loan svm
is_good_loan tree
is_high_credit logistic
is_high_credit gbm
is_high_credit nn
is_high_credit svm
is_high_credit tree
Running Baseline Experiment
Running Tasks Experiment
Running Models Experiment
Running Data Partitions Experiment
is_good_loan logistic
is_good_loan gbm
is_good_loan nn
is_good_loan svm
is_good_loan tree
is_high_credit logistic
is_high_credit gbm
is_high_credit nn



100%|█████████████████████████████████████████████| 9/9 [03:07<00:00, 20.84s/it]

is_high_credit svm
is_high_credit tree
